Niels Gadella S1156086

Importeer hier het model dat gesaved word in het siamese_network jupyter file
en de afbeelding die je wilt uploaden

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(8, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(4, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(64, activation='relu'))

In [ ]:
model = None

In [ ]:
model = tf.keras.models.load_model('model.keras')

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 8)         2312      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 8)         0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 4)         292       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 6, 6, 4)           0

In [ ]:
import cv2

afb1 = cv2.imread('afb1.jpg')
afb2 = cv2.imread('afb2.jpg')

print(afb1.shape)
print(afb2.shape)

afb1_scaled = cv2.resize(afb1, (64, 64))
afb2_scaled = cv2.resize(afb2, (64, 64))

print(afb1_scaled.shape)
print(afb2_scaled.shape)

afb1_scaled = afb1_scaled/255
afb2_scaled = afb2_scaled/255
afb1_scaled = afb1_scaled.reshape((1, 64, 64, 3))
afb2_scaled = afb2_scaled.reshape((1, 64, 64, 3))

emb1 = model.predict(afb1_scaled)
emb2 = model.predict(afb2_scaled)

(441, 413, 3)
(518, 621, 3)
(64, 64, 3)
(64, 64, 3)
1/1 [==============================] - 0s 34ms/step


In [ ]:
import numpy as np
from scipy import spatial

cosine_similarity = 1 - spatial.distance.cosine(emb1[0], emb2[0])
print(cosine_similarity)

0.8945448398590088


In [ ]:
from PIL import Image
import numpy as np

def convert_image_to_c_array(image_path, variable_name):
    # Load and resize
    img = Image.open(image_path)
    img = img.resize((64, 64))

    # Convert image to numpy array
    img_array = np.array(img, dtype=np.uint8)
    img_array = img_array.astype(np.float32) / 255.0
    # Flatten the array and convert to list
    img_list = img_array.flatten().tolist()

    # Create C array
    c_array = f"const ufloat32_t {variable_name}[] = {{" + ",".join(map(str, img_list)) + "};"

    return c_array

# Convert images
image1_c_array = convert_image_to_c_array("afb1.jpg", "image1_data")
image2_c_array = convert_image_to_c_array("afb2.jpg", "image2_data")

# Write to header files
with open("image1.h", "w") as f:
    f.write(image1_c_array)

with open("image2.h", "w") as f:
    f.write(image2_c_array)

In [ ]:
import numpy as np
# Create a representative dataset function
def representative_dataset():
    for _ in range(100):
        # Get sample input data as a numpy array in a method of your choosing
        data = np.random.rand(1, 64, 64, 3)
        yield [data.astype(np.float32)]

# Convert the model to TFLite format with full integer quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8  # or tf.int8
converter.inference_output_type = tf.uint8  # or tf.int8
tflite_quant_model = converter.convert()

with open('/content/model.tflite', 'wb') as f:
    f.write(tflite_quant_model)

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Model omzetten naar een header file die gebruikt kan worden door de arduino ide

In [ ]:
!xxd -i /content/model.tflite > model.h